# LEVERAGING AN AI AGENT FOR AUTOMATED INVOICE FINANCIAL CLASSIFICATION

I will use Gemini 2.0 Through OpenAI library <br>
Source: https://ai.google.dev/gemini-api/docs/openai?hl=pt-br

In [112]:
# Os modelos do Gemini podem ser acessados usando as bibliotecas OpenAI (Python e TypeScript / Javascript) com a API REST,
# atualizando três linhas de código e usando sua chave da API Gemini. Se você ainda não usa as bibliotecas OpenAI,
# recomendamos chamar a API Gemini diretamente.

## 1. Importing & API Key

In [113]:
from openai import OpenAI  # usado para o client estilo OpenAI(client)
from dotenv import load_dotenv
import os
import time
import pandas as pd

In [114]:
load_dotenv()
api_key = os.getenv("api_key")

In [115]:
# !pip install openai

In [116]:
# !pip install python-dotenv

In [117]:
# import openai
# import time
# from io import StringIO
# print(openai.__version__)

In [118]:
# from openai import OpenAI
# import openai

In [119]:
# from openai import OpenAI
# from dotenv import load_dotenv
# import os

# # Carrega as variáveis do .env
# load_dotenv()

# # Lê a variável de ambiente
# api_key = os.getenv("api_key")

In [120]:
#import pandas as pd
# !pip install pandas

## 2. Obtaining information from the invoice and preprocessing the dataset

In [121]:
df = pd.read_csv('data/Fatura2025-05-15.csv', sep=';')
df.head(1)

,Data,Estabelecimento,Portador,Valor
0,01/05/2025,MAE JOANA,FERNANDO CIRONE,"R$ 64,35"


In [122]:
df['Valor'] = (
    df['Valor']
    .str.replace('R$', '', regex=False)   # Remove "R$"
    .str.replace(',', '.', regex=False)   # Troca vírgula decimal por ponto
    .str.strip()                          # Remove espaços em branco
    .astype(float)                        # Converte para float
)

In [123]:
print(df.head())
print(df.dtypes)

         Data Estabelecimento         Portador  Valor
0  01/05/2025       MAE JOANA  FERNANDO CIRONE  64.35
1  01/05/2025     KATIA ROCHA  FERNANDO CIRONE  10.00
2  01/05/2025      UBER* TRIP  FERNANDO CIRONE  36.98
3  01/05/2025   UBER* PENDING  FERNANDO CIRONE  35.21
4  01/05/2025        METRO RJ  FERNANDO CIRONE   7.90
Data                object
Estabelecimento     object
Portador            object
Valor              float64
dtype: object


In [124]:
df.shape

(59, 4)

In [125]:
# # 2. Converter as primeiras linhas para texto
# gastos_texto = df.head(10).to_string(index=False)
# gastos_texto

In [126]:
# gastos_texto = df.to_string(index=False)
# gastos_texto

In [127]:
# # 3. Construir o prompt para classificar os gastos
# prompt = f"""
# I am a user with the following expenses extracted from my invoice:

# {gastos_texto}

# Can you classify each expense into categories such as Food, Transport, Leisure, Health, etc.?
# Please respond in table format with an additional column called "Category". Add a brief comment with your conclusion.
# Consider the following: Fernando Cirone is me, the user;

# """

## 3. Getting the Open AI library for latter using the Gemini model

In [128]:
client = OpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [129]:
# # === 3. Loop em blocos de 20 linhas ===
# bloco_size = 20

# for i in range(0, len(df), bloco_size):
#     bloco_df = df.iloc[i:i+bloco_size]
#     bloco_txt = bloco_df.to_string(index=False)

#     # Prompt personalizado com instruções
#     prompt = f"""
# I am a user with the following expenses extracted from my invoice:

# {bloco_txt}

# Can you classify each expense into categories such as Food, Transport, Leisure, Health, etc.?
# Please respond in table format with an additional column called 'Category'. Add a brief comment with your conclusion.

# Consider the following:
# 1) Fernando Cirone is me, the user;
# 2) Answer in English, even though the CSV content is written in Portuguese.
# """

#     # Enviar requisição à API Gemini
#     response = client.chat.completions.create(
#         model="gemini-2.0-flash",
#         messages=[
#             {"role": "system", "content": "You are a financial assistant specialized in classifying personal expenses."},
#             {"role": "user", "content": prompt}
#         ]
#     )

#     # Mostrar resultado do bloco
#     print(f"\n📄 Block {i // bloco_size + 1} (lines {i + 1} to {min(i + bloco_size, len(df))}):\n")
#     print(response.choices[0].message.content)

#     # Espera de 1 segundo para não sobrecarregar a API
#     time.sleep(20)

## 4. Creating a function to handle API responses, since I'm using the free model (which allows only a limited number of requests)

In [130]:
def tentar_enviar(prompt, tentativas=3, espera=20):
    #!parametros!
    # tentativas = a quantidade de tentativas ate dar erro
    # a espera ate dar erro novamente

    for tentativa in range(tentativas): #aqui ele tenta 3x
        try:
            response = client.chat.completions.create(
                model="gemini-2.0-flash",
                messages=[
                    {"role": "system", "content": "You are a financial assistant specialized in classifying personal expenses."},
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f" Error on try {tentativa + 1}: {e}")
            time.sleep(espera) #aqui ele da o sleep na quantidade de segundos
    return "Error After several times. Chunk jumped"

## 5. Creating a loop to use chunk sizes of the invoice 

In [131]:
len(df)

59

In [132]:
bloco_size = 20 #20 linhas!
df_classificados = [] #df to salve in dataframe


for i in range(0, len(df), bloco_size): # inicio, fim , passo
    bloco_df = df.iloc[i:i + bloco_size] # i + i +20
    bloco_txt = bloco_df.to_string(index=False)

    prompt = f"""
    You are a financial assistant specialized in classifying expenses.

    Below is a list of expenses from Fernando Cirone's invoice:

    {bloco_txt}

    Classify each expense into categories such as Food, Transport, Leisure, Health, etc.
    Return ONLY a valid CSV table using semicolon (;) as separator and dot (.) as decimal separator.
    Use the same columns as provided — 'Data', 'Estabelecimento', 'Portador', 'Valor' — and add one column named 'Categoria'.

    Do NOT include explanations, commentary or markdown formatting.
    Start directly with the header line.

    Example:
    Data;Estabelecimento;Portador;Valor;Categoria

    Answer in English, even though the table content is in Portuguese.
    """

    print(f"\n Processing block {i // bloco_size + 1} (lines {i + 1} to {min(i + bloco_size, len(df))})...")
    resposta = tentar_enviar(prompt)

    if resposta:
        try:
            # Tenta ler a resposta como CSV
            print("Sucess:\n")
            print(resposta)

            df_resposta = pd.read_csv(StringIO(resposta), sep=";")
            df_classificados.append(df_resposta)
        except Exception as e:
            print(f"erro ao ler como CSV: {e}")
    else:
        print("Bloco pulado por falhas consecutivas")

    time.sleep(5)


 Processing block 1 (lines 1 to 20)...
Sucess:

Data;Estabelecimento;Portador;Valor;Categoria
01/05/2025;MAE JOANA;FERNANDO CIRONE;64.35;Food
01/05/2025;KATIA ROCHA;FERNANDO CIRONE;10.00;Personal Care
01/05/2025;UBER* TRIP;FERNANDO CIRONE;36.98;Transport
01/05/2025;UBER* PENDING;FERNANDO CIRONE;35.21;Transport
01/05/2025;METRO RJ;FERNANDO CIRONE;7.90;Transport
01/05/2025;UBER * PENDING;FERNANDO CIRONE;15.80;Transport
01/05/2025;UBER * PENDING;FERNANDO CIRONE;43.31;Transport
01/05/2025;BAR URCA;FERNANDO CIRONE;19.00;Leisure
01/05/2025;UBER* PENDING;FERNANDO CIRONE;11.62;Transport
01/05/2025;BAR URCA;FERNANDO CIRONE;38.00;Leisure
02/05/2025;PB*SAMSUNG;FERNANDO CIRONE;29.90;Electronics
02/05/2025;CAM ON BOTAFOGO;FERNANDO CIRONE;127.50;Food
02/05/2025;BR*BR1FOODTOSAV;FERNANDO CIRONE;46.97;Food
02/05/2025;DROGARIA CRISTAL;FERNANDO CIRONE;30.98;Health
02/05/2025;UBER *TRIP HELP.UBER.COM;FERNANDO CIRONE;12.20;Transport
03/05/2025;UBER* PENDING;FERNANDO CIRONE;30.76;Transport
03/05/2025;SHOPE

In [133]:
# Junta todos os blocos classificados
df_final = pd.concat(df_classificados, ignore_index=True)
df_final.head(2)

,Data,Estabelecimento,Portador,Valor,Categoria
0,01/05/2025,MAE JOANA,FERNANDO CIRONE,64.35,Food
1,01/05/2025,KATIA ROCHA,FERNANDO CIRONE,10.00,Personal Care


In [134]:
# # # Salva o DataFrame final com a coluna Categoria
# df_final.to_csv('data/fatura_classificada.csv', index=False, sep=";")